In [2]:
from pyspark.sql import SparkSession
Spark = (SparkSession.builder.appName("capstone_ML")\
        .config("hive.metastore.uris","thrift://ip-10-1-2-24.ap-south-1.compute.internal:9083")\
        .enableHiveSupport().getOrCreate())

# Read all the tables

In [3]:
titles = Spark.sql("select * from mounica.titles")
departments =  Spark.sql("select * from mounica.Departments")
employees = Spark.sql("select * from mounica.employeesorg")
department_employees = Spark.sql("select * from mounica.Department_Employees")
department_managers = Spark.sql("select * from mounica.Department_Managers")
salaries = Spark.sql("select * from mounica.Salaries")

In [4]:
titles.show()

+--------+------------------+
|title_id|             title|
+--------+------------------+
|   e0001|Assistant Engineer|
|   e0002|          Engineer|
|   e0003|   Senior Engineer|
|   e0004|  Technique Leader|
|   m0001|Managerï»¿title_id|
|   s0001|             Staff|
|   s0002|      Senior Staff|
+--------+------------------+



In [5]:
departments.show()

+-------+------------------+
|dept_no|         dept_name|
+-------+------------------+
|   d001|         Marketing|
|   d002|           Finance|
|   d003|     HumanResource|
|   d004|        Production|
|   d005|       development|
|   d006|Quality Management|
|   d007|             Sales|
|   d008|          Research|
|   d009|  Customer Service|
+-------+------------------+



In [6]:
employees.show()

+------+------------+----------+----------+----------+---+----------+--------------+------------------------+--------+----------+
|emp_no|emp_title_id|birth_date|first_name| last_name|sex| hire_date|no_of_projects|last_performance_ratings|left_org| last_date|
+------+------------+----------+----------+----------+---+----------+--------------+------------------------+--------+----------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+------+------------+----------+----------+----------+---+----------+--------------+------------------------+--------+----------+
only showing top 20 rows



In [7]:
department_managers.show()

+-------+------+
|dept_no|emp_no|
+-------+------+
|   d001|110022|
|   d001|110039|
|   d002|110085|
|   d002|110114|
|   d003|110183|
|   d003|110228|
|   d004|110303|
|   d004|110344|
|   d004|110386|
|   d004|110420|
|   d005|110511|
|   d005|110567|
|   d006|110725|
|   d006|110765|
|   d006|110800|
|   d006|110854|
|   d007|111035|
|   d007|111133|
|   d008|111400|
|   d008|111534|
+-------+------+
only showing top 20 rows



In [10]:
department_employees.show()

+------+-------+
|emp_no|dept_no|
+------+-------+
| 10001|   d005|
| 10002|   d007|
| 10003|   d004|
| 10004|   d004|
| 10005|   d003|
| 10006|   d005|
| 10007|   d008|
| 10008|   d005|
| 10009|   d006|
| 10010|   d004|
| 10010|   d006|
| 10011|   d009|
| 10012|   d005|
| 10013|   d003|
| 10014|   d005|
| 10015|   d008|
| 10016|   d007|
| 10017|   d001|
| 10018|   d004|
| 10018|   d005|
+------+-------+
only showing top 20 rows



In [11]:
salaries.show()

+------+------+
|emp_no|salary|
+------+------+
| 10001| 60117|
| 10002| 65828|
| 10003| 40006|
| 10004| 40054|
| 10005| 78228|
| 10006| 40000|
| 10007| 56724|
| 10008| 46671|
| 10009| 60929|
| 10010| 72488|
| 10011| 42365|
| 10012| 40000|
| 10013| 40000|
| 10014| 46168|
| 10015| 40000|
| 10016| 70889|
| 10017| 71380|
| 10018| 55881|
| 10019| 44276|
| 10020| 40000|
+------+------+
only showing top 20 rows



# Join all the tables at employees level

In [40]:
df = employees.join(department_employees, on='emp_no', how='left')\
               .join(departments,departments.dept_no == department_employees.dept_no, how='left')\
               .join(department_managers, on='emp_no', how='left')\
               .join(salaries, on='emp_no', how='left')\
               .join(titles,employees.emp_title_id==titles.title_id, how='left')

In [41]:
df.toPandas()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,no_of_projects,last_performance_ratings,left_org,last_date,dept_no,dept_no,dept_name,dept_no,salary,title_id,title
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,2,A,0,\r,d002,d002,Finance,None,40000,s0001,Staff
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,1,C,0,\r,d004,d004,Production,None,53422,e0002,Engineer
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,9,A,0,\r,d004,d004,Production,None,48973,e0002,Engineer
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,2,C,0,\r,d003,d003,HumanResource,None,40000,s0001,Staff
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,6,B,0,\r,d006,d006,Quality Management,None,40000,e0003,Senior Engineer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331598,255832,e0002,5/8/1955,Yuping,Dayang,F,2/26/1995,10,A,0,\r,d004,d004,Production,None,75355,e0002,Engineer
331599,76671,s0001,6/9/1959,Ortrud,Plessier,M,2/24/1988,8,B,0,\r,d007,d007,Sales,None,61886,s0001,Staff
331600,264920,s0001,9/22/1959,Percy,Samarati,F,9/8/1994,1,B,0,\r,d007,d007,Sales,None,62772,s0001,Staff
331601,264920,s0001,9/22/1959,Percy,Samarati,F,9/8/1994,1,B,0,\r,d002,d002,Finance,None,62772,s0001,Staff


In [42]:
df.toPandas().head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
emp_no,473302,475053,57444,421786,282238,263976,273487,461591,477657,219881
emp_title_id,s0001,e0002,e0002,s0001,e0003,e0003,s0001,s0002,e0002,s0002
birth_date,7/25/1953,11/18/1954,1/30/1958,9/28/1957,10/28/1952,10/30/1959,4/14/1957,11/17/1964,12/18/1962,4/24/1956
first_name,Hideyuki,Byong,Berry,Xiong,Abdelkader,Eran,Christoph,Xudong,Lihong,Kwangyoen
last_name,Zallocco,Delgrande,Babb,Verhoeff,Baumann,Cusworth,Parfitt,Samarati,Magliocco,Speek
sex,M,F,F,M,F,M,M,M,M,F
hire_date,4/28/1990,9/7/1991,3/21/1992,11/26/1987,1/18/1991,11/14/1986,6/28/1991,11/13/1985,10/23/1993,2/14/1993
no_of_projects,2,1,9,2,6,8,10,2,5,6
last_performance_ratings,A,C,A,C,B,B,A,A,A,B
left_org,0,0,0,0,0,1,0,1,0,0


In [43]:
df1 = df

In [36]:
#Columns that will be used as features and their types
continuous_features = ['no_of_projects','salary']

categorical_features = ['last_performance_ratings', 'dept_name', 'left_org', 'title']

In [37]:
#Encoding all categorical features
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

In [38]:
def create_category( dataset, field_name ):
    idx_col = field_name + "Index"
    col_vec = field_name + "Vec"

    stringIndexer = StringIndexer( inputCol=field_name, outputCol=idx_col )

    model = stringIndexer.fit( dataset )
    indexed = model.transform( dataset )

    encoder = OneHotEncoder( dropLast=True,inputCol=idx_col,outputCol= col_vec )

    return encoder.transform( indexed )

In [44]:
for col in categorical_features:
    df1 = create_category(df1, col)
    df1.cache()

In [45]:
df1.columns

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date',
 'no_of_projects',
 'last_performance_ratings',
 'left_org',
 'last_date',
 'dept_no',
 'dept_no',
 'dept_name',
 'dept_no',
 'salary',
 'title_id',
 'title',
 'last_performance_ratingsIndex',
 'last_performance_ratingsVec',
 'dept_nameIndex',
 'dept_nameVec',
 'left_orgIndex',
 'left_orgVec',
 'titleIndex',
 'titleVec']

In [46]:
categorical_vecs = [ "".join( (cat, "Vec") ) for cat in categorical_features ]

In [47]:
categorical_vecs

['last_performance_ratingsVec', 'dept_nameVec', 'left_orgVec', 'titleVec']

In [48]:
#Including all features for model building

all_features= continuous_features + categorical_vecs

In [49]:
all_features

['no_of_projects',
 'salary',
 'last_performance_ratingsVec',
 'dept_nameVec',
 'left_orgVec',
 'titleVec']

In [50]:
# Creating the vector of all predictors
assembler = VectorAssembler( inputCols = all_features, outputCol = "features")

In [51]:
df1 = assembler.transform(df1)

In [53]:
# Setting the target variables - left
df1 = df1.withColumn( "label", df1.left_org.cast( 'int' ) )

In [54]:
df1.select( "features", "label" ).show( 5 )

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(21,[0,1,3,14,15]...|    0|
|(21,[0,1,4,7,14,1...|    0|
|(21,[0,1,3,7,14,1...|    0|
|(21,[0,1,4,13,14,...|    0|
|(21,[0,1,2,12,14,...|    0|
+--------------------+-----+
only showing top 5 rows



In [55]:
#Split the dataset
train_df, test_df = df1.randomSplit( [0.7, 0.3], seed = 42 )

In [56]:
#Train Linear Regression Model
from pyspark.ml.regression import LinearRegression

In [57]:
linreg = LinearRegression(maxIter=500, regParam=0.0)
lm = linreg.fit( train_df )

In [58]:
# Intercept and coefficients
lm.intercept

0.9999999999991521

In [59]:
lm.coefficients

DenseVector([-0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [60]:
#Making predictions on test data
y_pred  =  lm.transform( test_df )
y_pred.select( 'features', 'label', 'prediction' ).show( 5 )

+--------------------+-----+--------------------+
|            features|label|          prediction|
+--------------------+-----+--------------------+
|(21,[0,1,2,8,14,1...|    0|-1.11022302462515...|
|(21,[0,1,4,7,14,1...|    0|5.551115123125783...|
|(21,[0,1,3,7,14,1...|    0|3.330669073875469...|
|(21,[0,1,4,13,14,...|    0|-1.11022302462515...|
|(21,[0,1,4,8,14,1...|    0|-1.11022302462515...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [62]:
# Metrics to check the accuracy of model (goodness of fit)
#Calculating RMSE and R-Squared
from pyspark.ml.evaluation import RegressionEvaluator

In [63]:
rmse_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="rmse" )

In [64]:
lm_rmse = rmse_evaluator.evaluate( y_pred )
lm_rmse

6.6320290966517706e-15

In [65]:
r2_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="r2" )
lm_r2 = r2_evaluator.evaluate( y_pred )
lm_r2

1.0

In [66]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

## Create indexers for the categorical features

In [68]:

indexers = [StringIndexer(inputCol=c, outputCol="{}_idx".format(c)) for c in categorical_features]

## encode the categorical features

In [69]:
encoders = [ OneHotEncoder(
      inputCol=idx.getOutputCol(),
      outputCol="{0}_enc".format(idx.getOutputCol())) for idx in indexers]

## Create vectors for all features categorical and continuous


In [70]:
assembler = VectorAssembler(
  inputCols=[enc.getOutputCol() for enc in encoders] + continuous_features,
  outputCol="features")

## Initialize the linear model

In [72]:
lrModel = LinearRegression( maxIter = 10 )

## Create the pipeline with sequence of activities

In [74]:
pipeline = Pipeline( stages= [indexers, encoders, assembler, lrModel ])